# Read and visualise CoLoRe's output
This is a notebook to read example output from CoLoRe and make some basic plots showing the QSO distributions and the skewers.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt

from lyacolore import utils

In [ ]:
# Define some plotting parameters: the figure size, font size and some nice colours
figsize = (12,8)
fontsize = 18; plt.rc('font', size=fontsize)
colours = C0,C1,C2,C3 = ['#F5793A','#A95AA1','#85C0F9','#0F2080']

## Open an example data file and show its structure.
Here, there is a choice between a Gaussian output file, or a 2LPT (density) one. Set the variable `file_type` to `gaussian` or `2lpt` to make your choice.

The HDUs contain the following information:

 - `CATALOG` contains information about the QSOs.
 - `DENSITY` contains the density skewers. Skewers are stored as an array, where each row corresponds to a separate skewer.
 - `VELOCITY` contains the velocity skewers. Skewers are stored as an array, where each row corresponds to a separate skewer.
 - The 4th HDU contains cosmological quantities (redshift, comoving distance etc) along the skewers.

In [ ]:
file_type = 'gaussian'

In [ ]:
assert file_type in ['gaussian','2lpt']

In [ ]:
hdulist = fits.open(os.environ['LYACOLORE_PATH']
                    +'/example_data/{}/colore_output/out_srcs_s1_0.fits'.format(file_type))
hdulist.info()

## Take a look at the QSOs in the file.
We first extract the information about the QSOs in this file, looking at their distribution on the sky and their number density distribution.

In [ ]:
z_qso = hdulist[1].data['Z_COSMO']
N_qso = len(z_qso)
print('INFO: There are {} quasars in the sample.'.format(N_qso))

In [ ]:
# Plot the locations of the quasars.
fig, axs = plt.subplots(1,1,figsize=figsize,squeeze=False)
RA = hdulist[1].data['RA']
DEC = hdulist[1].data['DEC']
phi = RA*np.pi/180
theta = np.pi/2 - DEC*np.pi/180
axs[0,0].scatter(phi/np.pi,np.cos(theta),color=C0)
axs[0,0].set_xlim(0.0,2.0)
axs[0,0].set_ylim(-1.0,1.0)
axs[0,0].set_xlabel(r'$\phi/\pi$')
axs[0,0].set_ylabel(r'$\cos(\theta)$')
plt.show()

In [ ]:
# Plot the n(z) distribution of the quasars.
fig, axs = plt.subplots(1,1,figsize=figsize,squeeze=False)
bins = np.linspace(1.0,4.0,31)
zqso = hdulist[1].data['Z_COSMO']
axs[0,0].hist(zqso,bins=bins,histtype='step',color=C0)
axs[0,0].set_xlabel(r'$z$')
axs[0,0].set_ylabel('# QSOs')
plt.show()

## Extract some general data from the file.
We now extract data from the file useful for plotting individual skewers shortly

In [ ]:
# Extract redshift from data file
z = hdulist[4].data['Z']
z = np.asarray(z)

In [ ]:
# Extract the delta skewers from the file, and make a mask for them according to a maximum rest frame wavelength.
lr_max = 1200.
delta_skewers = hdulist[2].data
mask = utils.make_IVAR_rows(lr_max,z_qso,np.log10(utils.lya_rest*(1+z)))

## Look at an individual skewer.

In [ ]:
# Get the skewer for the highest redshift quasar in the sample.
ind = np.argmax(hdulist[1].data['Z_COSMO'])
delta = delta_skewers[ind,:]

In [ ]:
#Show delta vs z
fig, axs = plt.subplots(1,1,figsize=figsize,squeeze=False)
mean_delta = np.average(delta,weights=mask[ind])
print('INFO: mean delta =', mean_delta)
mean_delta_squared = np.average(delta**2,weights=mask[ind])
sigma_delta = np.sqrt(mean_delta_squared-mean_delta**2)
print('INFO: std delta =', sigma_delta)
axs[0,0].plot(z,delta,c=C0)
axs[0,0].set_xlabel('z')
axs[0,0].set_ylabel('$\\delta$')
plt.show()

## Look at the mean and std of all skewers in the file.
For Gaussian skewers, this should just be noise around zero. For 2LPT or lognormal skewers it will be non-zero.

In [ ]:
# Calculate the mean delta as a function of redshift.
overall_mean_delta = np.average(delta_skewers,weights=mask)
print('mean delta over all skewers =', overall_mean_delta)
w = mask.sum(axis=0)>0

In [ ]:
# Plot the mean delta as a function of redshift.
fig, axs = plt.subplots(1,1,figsize=figsize,squeeze=False)
axs[0,0].plot(z[w],np.average(delta_skewers[:,w],weights=mask[:,w],axis=0),c=C0)
axs[0,0].set_xlabel('z')
axs[0,0].set_ylabel('$<\\delta>$')
plt.axhline(y=overall_mean_delta,c='grey')
plt.show()

In [ ]:
# Calculate the standard deviation as a function of redshift.
overall_mean_delta = np.average(delta_skewers,weights=mask)
overall_mean_delta_squared = np.average(delta_skewers**2,weights=mask)
overall_sigma_delta = np.sqrt(overall_mean_delta_squared-overall_mean_delta**2)
print(r'INFO: std over all skewers = {:1.4f}'.format(overall_sigma_delta))
w = mask.sum(axis=0)>0
mean_delta = np.average(delta_skewers[:,w],weights=mask[:,w],axis=0)
mean_delta_squared = np.average(delta_skewers[:,w]**2,weights=mask[:,w],axis=0)

In [ ]:
# Plot the standard deviation as a function of redshift.
fig, axs = plt.subplots(1,1,figsize=figsize,squeeze=False)
axs[0,0].plot(z[w],np.sqrt(mean_delta_squared-mean_delta**2),c=C0)
axs[0,0].set_xlabel('z')
axs[0,0].set_ylabel('$\sigma_{\delta}$')
plt.axhline(y=overall_sigma_delta,c='grey')
plt.show()

## Look at the PDF of the skewers in redshift bins.

Gaussian skewers have no redshift evolution and so all redshift bins should have identical, Gaussian PDFs. 2LPT skewers will show some redshift evolution, and will represent physical deltas (i.e. values between -1 and inf).

In [ ]:
# Plot the pdf of deltas in redshift bins.
z_bins = [(0,1),(1,2),(2,3),(3,)]

fig, axs = plt.subplots(1,1,figsize=figsize,squeeze=False)
if file_type == 'gaussian':
    d_bins = np.linspace(-5,5,100)
elif file_type == '2lpt':
    d_bins = np.linspace(-1,4,100)

for i,zbin in enumerate(z_bins):    
    if len(zbin)==2:
        w = ((z>zbin[0]) * (z<zbin[1]))
        label = r'${}<z<{}$'.format(zbin[0], zbin[1])
    else:
        w = ((z>zbin[0]))
        label = r'${}<z$'.format(zbin[0])
    axs[0,0].hist(np.ravel(hdulist[2].data[:,w]),bins=d_bins,weights=np.ravel(mask[:,w]),
             density=True,histtype='step',label=label,color=colours[i%len(colours)])
axs[0,0].set_xlabel('$\\delta$')
axs[0,0].set_ylabel('$P(\delta)$')
plt.legend()
plt.show()